# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

import sys
import re
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
import pandas as pd
import pickle
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ekaap\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ekaap\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ekaap\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
# load data from database
engine = create_engine('sqlite:///disaster.db')
df = pd.read_sql_table('disaster', engine)
X = df['message']
y = df.iloc[:, 4:]
y.replace({'related': 2}, 1, inplace = True)
category_names = list(y.columns)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# train classifier
pipeline.fit(X_train, y_train)

Pipeline(steps=[('text_pipeline',
                 Pipeline(steps=[('vect',
                                  CountVectorizer(tokenizer=<function tokenize at 0x25D3AAD8>)),
                                 ('tfidf', TfidfTransformer())])),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pd.DataFrame(pipeline.predict(X_test))
y_pred.columns = category_names
print(classification_report(y_test, y_pred, target_names=category_names))

                        precision    recall  f1-score   support

               related       0.67      0.96      0.79      2034
               request       0.48      0.09      0.14      1080
                 offer       0.00      0.00      0.00         4
           aid_related       0.52      0.14      0.22      1189
          medical_help       0.00      0.00      0.00       173
      medical_products       0.00      0.00      0.00       104
     search_and_rescue       0.00      0.00      0.00        64
              security       0.00      0.00      0.00        40
              military       0.00      0.00      0.00        13
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       226
                  food       0.33      0.00      0.01       438
               shelter       0.06      0.00      0.01       330
              clothing       0.00      0.00      0.00        24
                 money       0.00      

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
parameters = {
        'text_pipeline__tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [25, 50]
    }

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('text_pipeline',
                                        Pipeline(steps=[('vect',
                                                         CountVectorizer(tokenizer=<function tokenize at 0x25D3AAD8>)),
                                                        ('tfidf',
                                                         TfidfTransformer())])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [25, 50],
                         'text_pipeline__tfidf__use_idf': (True, False)})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
y_pred = pd.DataFrame(cv.predict(X_test))
y_pred.columns = category_names
print(classification_report(y_test, y_pred, target_names=category_names))

                        precision    recall  f1-score   support

               related       0.67      0.95      0.78      2034
               request       0.47      0.07      0.13      1080
                 offer       0.00      0.00      0.00         4
           aid_related       0.53      0.12      0.20      1189
          medical_help       0.00      0.00      0.00       173
      medical_products       1.00      0.01      0.02       104
     search_and_rescue       0.00      0.00      0.00        64
              security       0.00      0.00      0.00        40
              military       0.00      0.00      0.00        13
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       226
                  food       0.31      0.01      0.02       438
               shelter       0.12      0.01      0.01       330
              clothing       0.00      0.00      0.00        24
                 money       0.00      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [10]:
model_filepath = 'saved_model'
pickle.dump(cv, open(model_filepath, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.